In [66]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors # Use this is for creating a cursor-interactive plot with "%matplotlib notebook"
from sklearn.decomposition import NMF # Use this for training Non-negative Matrix Factorization
from sklearn.utils.extmath import randomized_svd # Use this for training Singular Value Decomposition
from sklearn.manifold import TSNE # Use this for training t-sne manifolding

In [67]:
dir = './최종데이터/'
df_product = pd.read_csv(dir + 'basic_data_edit.csv', usecols=['00.상품_URL','00.상품코드','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음'],encoding='cp949')
df_review = pd.read_csv(dir + 'review_data_edit.csv', usecols=['00.상품코드','01.리뷰어 닉네임','02_1.피부 타입','02_2.피부 톤','02_3.피부 고민','03.리뷰 평점','04.피부타입 | 피부고민 | 자극도','05.리뷰'],encoding='cp949')
print('done')

done


In [68]:
product_list = df_review['00.상품코드']
product_list = product_list.unique()
len(product_list)

430

In [69]:
len(df_product)*1000

703000

In [70]:
len(df_review)

158590

In [71]:
#column 이름 변경 및 결측치 제거
df_review.columns = ['code','user','type','tone','problem','rating','feature','review']
df_review = df_review.dropna(axis=0)

In [164]:
#column 이름 변경 및 결측치 제거
df_product.columns = ['00.상품_URL','code','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음']
df_product = df_product.dropna(axis=0)
df_product = df_product.drop_duplicates(['code'])

In [165]:
len(df_product)

522

In [73]:
df_review

,code,user,type,tone,problem,rating,feature,review
0,A000000147339,요다,건성,웜톤,민감성 | 아토피 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",개인적으로 성분으로 마케팅하는 제품들을 그렇게 좋아하진 않습니다. 특히 토너는 순한...
1,A000000147339,올리브양,복합성,쿨톤,모공 | 민감성 | 주름 | 탄력 | 트러블 | 홍조,5.0,"('건성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",복합성 피부에 많이 민감한 피부입니다. 요즘 봄 바람에 아침 저녁으로 기온차가 심해...
2,A000000147339,뚜비이,건성,쿨톤,민감성 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",지난번 올영체험단으로 제공받아 사용해보고 너무 좋아서 다시 구매했습니당!\r\n저는...
3,A000000147339,시래우,복합성,겨울쿨톤,다크서클 | 모공 | 미백 | 민감성 | 블랙헤드 | 아토피 | 잡티 | 주름 | ...,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",날씨 더워지면 광대가 뜨거워지면서 내 얼굴이 호빵맨이 되어가는 것 같고 좀 더 더워...
5,A000000147339,효지니,복합성,쿨톤,민감성 | 블랙헤드 | 아토피 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",요즘에 피부가 너무 나빠져서 잘 맞는 화장품 찾아볼려고 이것저것 구매를 해봤어요! ...
...,...,...,...,...,...,...,...,...
158580,A000000148095,겨울별,지성,쿨톤,각질 | 미백 | 블랙헤드 | 잡티 | 주름 | 트러블,5.0,"('지성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",봄철에딱인 속건조토너 흑미탄탄토너를 일주일정도 써보았습니다. 뷰티유튜버가 왜추천을했...
158582,A000000148095,moon219,복합성,쿨톤,모공 | 트러블,5.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",제품이 순하기도 하고 성분이 좋아서 그런지 토너팩을 매일같이 하니까 정말 많이 진정...
158583,A000000148095,리파리해파리파리,건성,웜톤,각질 | 모공 | 미백 | 블랙헤드 | 잡티 | 주름 | 탄력 | 트러블,4.0,"('복합성에 좋아요 | 주름/미백에 좋아요 | 자극없이 순해요',)",저는 올리브영체험단을 몇번하면서\r\n항상 제품과 리뷰작성바란다는 안내문만 받아보다...
158586,A000000148095,luv9,건성,웜톤,각질 | 민감성 | 주름 | 홍조,4.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",체험단 신청해서 받아보게 되었는데! 직접 손편지로 리뷰작성안내 해주신 브랜드는 처음...


In [74]:
user_list = df_review['code']
user_list = user_list.unique()
user_list[0]
df_review_user = df_review[df_review['code']==user_list[0]]

problem_dict = {}
for problem in df_review_user['problem']:
    problem_list = problem.split('|')
    for i in problem_list:
        i = i.replace(" ","")
        if i in problem_dict:
            problem_dict[i] += 1
        else:
            problem_dict[i] = 1
problem_dict_sort = sorted(problem_dict.items(),key = lambda x: x[1],reverse = True)
problem_dict_sort[0][0]


'트러블'

In [170]:
user_list = df_review['code']
user_list = user_list.unique()
user_list[0]
df_review_user = df_review[df_review['code']==user_list[0]]

#최다빈출 type
type_freq = df_review_user['type'].value_counts()
#최다빈출 tone
tone_freq = df_review_user['tone'].value_counts()
#최다빈출 problem
problem_dict = {}
for problem in df_review_user['problem']:
    problem_list = problem.split('|')
    for i in problem_list:
        i = i.replace(" ","")
        if i in problem_dict:
            problem_dict[i] += 1
        else:
            problem_dict[i] = 1
problem_dict_sort = sorted(problem_dict.items(),key = lambda x: x[1],reverse = True)



type = type_freq.index[0]
tone = tone_freq.index[0]
problem = ''
for i, problems in enumerate(problem_dict_sort):
    problem += problems[0] + ' '
    if i==20:
        break
        
        
print(type)
print(tone)
print(type+' '+tone+' '+problem)


복합성
웜톤
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 미백 홍조 탄력 각질 피지과다 주름 다크서클 아토피 


In [153]:
df_feature = pd.DataFrame(columns = ['code','feature'])

In [154]:
product_list = df_review['code']
product_list = product_list.unique()
index = 0
for product in product_list:
    df_review_user = df_review[df_review['code']==product]
    
    #최다빈출 type
    type_freq = df_review_user['type'].value_counts()
    #최다빈출 tone
    tone_freq = df_review_user['tone'].value_counts()
    #최다빈출 problem
    problem_dict = {}
    for problem in df_review_user['problem']:
        problem_list = problem.split('|')
        for i in problem_list:
            i = i.replace(" ","")
            if i in problem_dict:
                problem_dict[i] += 1
            else:
                problem_dict[i] = 1
    problem_dict_sort = sorted(problem_dict.items(),key = lambda x: x[1],reverse = True)



    type = type_freq.index[0]
    tone = tone_freq.index[0]
    problem = ''
    for i, problems in enumerate(problem_dict_sort):
        problem += problems[0] + ' '
        if i==4:
            break

    feature = type+' '+tone+' '+problem
    
    print(index)
    print(product)
    print(feature)
    
    df_feature.loc[str(index)] = [product,feature]
    
    index += 1

0
A000000147339
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 
1
A000000137180
복합성 웜톤 트러블 모공 블랙헤드 잡티 민감성 
2
A000000141704
복합성 웜톤 트러블 모공 잡티 블랙헤드 민감성 
3
A000000104007
복합성 웜톤 모공 트러블 잡티 민감성 블랙헤드 
4
A000000146589
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 
5
A000000016686
복합성 웜톤 모공 잡티 트러블 민감성 블랙헤드 
6
A000000161508
복합성 웜톤 모공 트러블 블랙헤드 잡티 각질 
7
A000000145578
복합성 웜톤 모공 트러블 잡티 민감성 블랙헤드 
8
A000000126862
복합성 쿨톤 트러블 모공 블랙헤드 민감성 잡티 
9
A000000156512
복합성 웜톤 잡티 트러블 모공 블랙헤드 민감성 
10
A000000150946
복합성 웜톤 트러블 모공 잡티 민감성 블랙헤드 
11
A000000148899
복합성 쿨톤 민감성 트러블 모공 잡티 주름 
12
A000000129008
복합성 웜톤 민감성 모공 트러블 블랙헤드 잡티 
13
A000000147524
복합성 웜톤 트러블 모공 블랙헤드 잡티 민감성 
14
A000000154177
복합성 웜톤 트러블 모공 블랙헤드 각질 잡티 
15
A000000157294
복합성 웜톤 모공 잡티 블랙헤드 민감성 주름 
16
A000000143827
복합성 웜톤 모공 트러블 잡티 각질 민감성 
17
A000000117541
복합성 쿨톤 모공 잡티 트러블 민감성 블랙헤드 
18
A000000012809
복합성 웜톤 민감성 잡티 트러블 모공 홍조 
19
A000000147387
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 
20
A000000010474
복합성 웜톤 모공 민감성 트러블 블랙헤드 잡티 
21
A000000147313
복합성 웜톤 블랙헤드 모공 잡티 트러블 미백 
22
A000000106421
복합성 웜톤 모공 블랙헤드 잡티 트러블 민감성 
23
A0000

234
A000000124493
복합성 웜톤 각질 잡티 트러블 블랙헤드 모공 
235
A000000143199
지성 웜톤 트러블 모공 미백 잡티 피지과다 
236
A000000124495
복합성 웜톤 잡티 모공 탄력 주름 블랙헤드 
237
A000000143228
복합성 웜톤 트러블 모공 민감성 블랙헤드 잡티 
238
A000000143458
복합성 웜톤 민감성 모공 트러블 잡티 블랙헤드 
239
A000000124326
건성 웜톤 민감성 블랙헤드 잡티 
240
A000000124194
복합성 웜톤 미백 아토피 잡티 주름 탄력 
241
A000000143970
건성 웜톤 잡티 민감성 트러블 각질 모공 
242
A000000143244
지성 웜톤 모공 트러블 
243
A000000138980
복합성 웜톤 트러블 잡티 모공 미백 블랙헤드 
244
A000000124496
건성 쿨톤 미백 잡티 주름 탄력 블랙헤드 
245
A000000124501
건성 웜톤 잡티 주름 민감성 탄력 각질 
246
A000000141718
복합성 웜톤 각질 트러블 모공 블랙헤드 
247
A000000124499
건성 웜톤 모공 민감성 블랙헤드 트러블 
248
A000000142945
복합성 웜톤 트러블 모공 탄력 블랙헤드 각질 
249
A000000142184
복합성 웜톤 모공 트러블 잡티 민감성 블랙헤드 
250
A000000142412
건성 쿨톤 홍조 트러블 민감성 피지과다 각질 
251
A000000142413
복합성 웜톤 모공 블랙헤드 잡티 트러블 민감성 
252
A000000142618
복합성 웜톤 모공 잡티 트러블 민감성 블랙헤드 
253
A000000142744
복합성 웜톤 민감성 잡티 모공 주름 탄력 
254
A000000142746
복합성 쿨톤 모공 트러블 잡티 홍조 민감성 
255
A000000138757
복합성 웜톤 잡티 트러블 모공 미백 민감성 
256
A000000138702
지성 웜톤 각질 모공 미백 블랙헤드 트러블 
257
A000000135773
복합성 웜톤

In [155]:
df_feature

,code,feature
0,A000000147339,복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드
1,A000000137180,복합성 웜톤 트러블 모공 블랙헤드 잡티 민감성
2,A000000141704,복합성 웜톤 트러블 모공 잡티 블랙헤드 민감성
3,A000000104007,복합성 웜톤 모공 트러블 잡티 민감성 블랙헤드
4,A000000146589,복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드
...,...,...
369,A000000122212,건성 웜톤 잡티 미백 주름 각질 홍조
370,A000000148253,건성 웜톤 홍조 모공 잡티 민감성 각질
371,A000000122094,복합성 쿨톤 모공 블랙헤드 주름 잡티 탄력
372,A000000121274,복합성 웜톤 민감성 트러블 모공 잡티 홍조


In [156]:
df_feature
df_feature.loc[str(index)] = ('input','건성 웜톤 트러블 미백 주름')

In [103]:
df_feature.to_csv(dir+'df_feature.csv',header = True, index = True,encoding='cp949')

In [157]:
from sklearn.feature_extraction.text import CountVectorizer

counter_vector = CountVectorizer(ngram_range=(1,3))
c_vector_features = counter_vector.fit_transform(df_feature['feature'])
c_vector_features.shape

(375, 700)

In [158]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_feature = cosine_similarity(c_vector_features,c_vector_features).argsort()[:,::-1]
print(similarity_feature)
print(similarity_feature.shape)

[[  0   4  48 ...  51 233 182]
 [ 33   1  66 ...  51 233 182]
 [343 341   2 ...  51 233 182]
 ...
 [372 366 271 ...  51 367 182]
 [373 351 260 ... 250  51 182]
 [374 180 364 ... 107 223 338]]
(375, 375)


In [159]:
def recommend_product_list(df_feature, code , top=11):
    #특정 영화정보 뽑아내기
    target_feature_index = df_feature[df_feature['code'] == code].index.values

    #타켓영화와 비슷한 코사인 유사도값
    sim_index = similarity_feature[int(target_feature_index),:top].reshape(-1)
    #본인제외
    sim_index = sim_index[sim_index!=target_feature_index]
    
    #추천 결과 새로운 df 생성, 평균평점(score)으로 정렬
    result = df_feature.iloc[sim_index]
    
    return result

In [166]:
df_result = recommend_product_list(df_feature, code='input')

In [167]:
df_result

,code,feature
374,input,건성 웜톤 트러블 미백 주름
180,A000000011084,건성 웜톤 트러블 잡티 블랙헤드 미백 민감성
364,A000000148417,건성 웜톤 트러블 모공 블랙헤드 각질 잡티
57,A000000159291,건성 웜톤 민감성 모공 잡티 트러블 미백
369,A000000122212,건성 웜톤 잡티 미백 주름 각질 홍조
214,A000000141310,건성 웜톤 트러블 모공 블랙헤드 각질 민감성
360,A000000149231,건성 웜톤 잡티 주름
212,A000000141252,건성 웜톤 탄력
187,A000000163512,건성 웜톤 각질
168,A000000156725,복합성 웜톤 주름 블랙헤드 잡티 트러블 미백


In [168]:
df_recommend = pd.merge(df_product,df_result,on='code',how='right')
df_recommend['02.상품명']

0                                NaN
1         엠도씨 릴리프 토닉 위드 애프터셰이브 150ml
2              이너프 프로젝트 에센셜 로션 150ml
3                마몽드 레티놀 앰플 토너 250ml
4      다슈 맨즈 아쿠아 리얼 모이스트 올인원크림 153ml
5       [디오디너리] 비타민 씨 서스펜션 30% 인 실리콘
6      오우썸 수딩앤딥모이스처라이징 카렌듈라 토너 200ml
7       진저6 라이크 화이트 블라썸 에센스 토너 150ml
8          [증량]식물나라 카렌둘라 진정 토너 500ml
9     아이디얼 포맨 빙하수 모공 애프터쉐이브 스킨 300ml
10        닥터디퍼런트 스케일링 토너 200ml (중건성)
Name: 02.상품명, dtype: object